In [1]:
import os, random, string, re
from io import StringIO
import numpy as np
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_sm")

In [2]:
# BIO tagging 

def tag_terms(doc, spans):
    
    is_last_term = False
    tagged = ""
    
    for sent in doc.sents:
        for token in sent:
            
            if not token.text or token.text.isspace():
                continue

            is_term = any([token in span for span in spans])
            tag = 'O'

            if is_term:
                if is_last_term:
                    tag = 'I'
                else:
                    tag = 'B'
                is_last_term = True
            else:
                is_last_term = False

            tagged += token.text + "\t" + tag + "\n"
        tagged += "\n"
        
    return tagged

In [3]:
special_cases = {}
prefix_re = re.compile('^[\W_]{1}')
suffix_re = re.compile('[\W_]{1}$')
infix_re = re.compile('[\W_]{1}')

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, rules=special_cases,
                                prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer)

In [4]:
nlp.tokenizer = custom_tokenizer(nlp)

In [7]:
gold_standard = pd.read_csv("gold-standard.csv").drop(["FWORD", "CODE"], axis=1)

new_labels = pd.read_csv("new-labels.csv")
new_labels.head(10)

gold_standard = gold_standard.append(new_labels)
gold_standard.head(10)

autism_terms = set(gold_standard["TEXT"])
print(autism_terms)

matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in autism_terms]
matcher.add("AutismTerms", None, *patterns)

{'parallel play', 'did initiate contact', 'hair flicking', 'does not seek out contact with other children', 'preoccupation with television', 'no interest in playing with others', 'social communication disorder', 'repeating sounds in perseverative way', 'impairment of peer relationships', 'engaging in some repetitive behavior', 'mostly watches other children', 'interacting socially with others', 'great deal of echolalia', 'picking at wallpaper until whole room is free of wallpaper', 'Hand movements', 'repetitive spinning of whole body', 'arm biting', 'limited interaction with other individuals', 'did show evidence of initiating an interaction', 'speech tone high-pitched', 'twirling and spinning', 'licking carpet during exam', 'toe-walking gait', 'could not be engaged in social interaction', 'no spontaneous social interaction', ' did not intiate any eye contact', 'hypersensitive to stimuli', 'facial expression is lacking for any social interchange', 'play is limited to a few preferred it

In [8]:
#INPUT_DIRECTORY= "pubmed_all_fulltext_input"
INPUT_DIRECTORY = "pubmed_abstracts_24860"

autism_texts = []
for filename in os.listdir(INPUT_DIRECTORY):
    if filename.endswith(".txt"):
        autism_texts.append(filename)

In [9]:
len(autism_texts)

20424

In [10]:
print("before", autism_texts[:5])
random.Random(777).shuffle(autism_texts) # shuffle abstracts and then divide to training, evaluation, testing
print("after shuffle", autism_texts[:5])

before ['28770039.txt', '19360649.txt', '9826299.txt', '1918227.txt', '16163569.txt']
after shuffle ['12164013.txt', '27121242.txt', '22917121.txt', '15995945.txt', '30145834.txt']


In [11]:
#directory_train_test = "full_text_train_test"
directory_train_test = "abstract_train_test"

In [ ]:
# 60% train, 20% dev, 20% test

# num_train = 12424 
# num_dev = 4000
# num_test = 4000

num_train = 325
num_dev = 110
num_test = 110

with open(os.path.join(directory_train_test, "train_test_split.txt"), "w") as f:
    
    for i, filename in enumerate(autism_texts):
        if filename.endswith(".txt"):

            # train 
            if i < num_train:
                f.write(filename + "\t" + "train" + "\n")

            # dev
            elif i < num_train + num_dev:
                f.write(filename + "\t" + "devel" + "\n")

            # test
            else:
                f.write(filename + "\t" + "test" + "\n")


In [ ]:
# check 

train_count = 0
dev_count = 0
test_count = 0

# check
with open(os.path.join(directory_train_test, "train_test_split.txt"), "r") as f:
    for line in f:
        if "train" in line:
            train_count = train_count + 1
        elif "devel" in line:
            dev_count = dev_count + 1
        elif "test" in line:
            test_count = test_count +1
        else:
            raise Exception("wrong type")
            
print("train_count", train_count)
print("dev_count", dev_count)
print("test_count", test_count)

In [ ]:
# num_train = 300
# num_dev = 200
# num_test = 46

with open(os.path.join(directory_train_test, "train_test_split.txt"), "r") as f:
    
    for i, line in enumerate(f, 1):
        
        filename = line.split("\t")[0].strip()
        typ = line.split("\t")[1].strip()
        
        path = os.path.join(INPUT_DIRECTORY, filename)
        
        # tag entities in the full-text
        with open(path, "r") as f:
            data = f.read()
        
        # get NER labels and do BIO tagging
        doc = nlp(data)

        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            span = doc[start:end]
            spans.append(span)
            
        tagged = tag_terms(doc, spans)
    
        # write tags to text file
        with open(os.path.join(directory_train_test, typ + ".tsv"), "a+") as f:
            f.write(tagged)
               
        if i % 1 == 0:
            print(typ, i, filename)

In [12]:
# num_train = 300
# num_dev = 200
# num_test = 46

# write test.tsv with paper info

with open(os.path.join(directory_train_test, "train_test_split.txt"), "r") as f:
    
    for i, line in enumerate(f, 1):
        
        filename = line.split("\t")[0].strip()
        typ = line.split("\t")[1].strip()
        
        path = os.path.join(INPUT_DIRECTORY, filename)
        
        # tag entities in the full-text
        with open(path, "r") as f:
            data = f.read()
        
        # get NER labels and do BIO tagging
        doc = nlp(data)

        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            span = doc[start:end]
            spans.append(span)
            
        tagged = tag_terms(doc, spans)
    
        # write tags to text file
        with open(os.path.join(directory_train_test, typ + "_w_paper.tsv"), "a+") as f:
            f.write(filename + "\n")
            f.write(tagged)
               
        if i % 10 == 0:
            print(typ, i, filename)

train 1 12164013.txt
train 2 27121242.txt
train 3 22917121.txt
train 4 15995945.txt
train 5 30145834.txt
train 6 18685284.txt
train 7 22548805.txt
train 8 29843508.txt
train 9 23060671.txt
train 10 14533392.txt
train 11 19138551.txt
train 12 18787937.txt
train 13 26436185.txt
train 14 26206232.txt
train 15 28303420.txt
train 16 20592203.txt
train 17 29040924.txt
train 18 18600439.txt
train 19 9502205.txt
train 20 28920185.txt
train 21 25641004.txt
train 22 26290361.txt
train 23 27343887.txt
train 24 17427189.txt
train 25 30443698.txt
train 26 17290335.txt
train 27 25744543.txt
train 28 23732106.txt
train 29 27789404.txt
train 30 27063700.txt
train 31 27004599.txt
train 32 21943103.txt
train 33 22545635.txt
train 34 30381235.txt
train 35 11442855.txt
train 36 19863947.txt
train 37 6085953.txt
train 38 29118348.txt
train 39 17280796.txt
train 40 11936551.txt
train 41 30284229.txt
train 42 23333377.txt
train 43 18556898.txt
train 44 28492151.txt
train 45 7592251.txt
train 46 18956241.txt


train 370 26257243.txt
train 371 30392026.txt
train 372 27255217.txt
train 373 22987915.txt
train 374 21316919.txt
train 375 28218269.txt
train 376 28500650.txt
train 377 10641305.txt
train 378 18850262.txt
train 379 15820630.txt
train 380 28301091.txt
train 381 24237479.txt
train 382 27899716.txt
train 383 12908830.txt
train 384 14681917.txt
train 385 26869267.txt
train 386 12757361.txt
train 387 16455174.txt
train 388 27435022.txt
train 389 28867142.txt
train 390 14594333.txt
train 391 19166581.txt
train 392 25518824.txt
train 393 9690939.txt
train 394 10478730.txt
train 395 19495952.txt
train 396 6274037.txt
train 397 27062034.txt
train 398 22917124.txt
train 399 10077303.txt
train 400 16566949.txt
train 401 29651777.txt
train 402 26248474.txt
train 403 30873514.txt
train 404 25245351.txt
train 405 24735654.txt
train 406 26373767.txt
train 407 19504361.txt
train 408 20637245.txt
train 409 22985783.txt
train 410 29717196.txt
train 411 17546488.txt
train 412 20102259.txt
train 413 263

train 730 28838582.txt
train 731 16648374.txt
train 732 29558884.txt
train 733 1378050.txt
train 734 23638906.txt
train 735 27247853.txt
train 736 15909405.txt
train 737 27153221.txt
train 738 26696102.txt
train 739 19307255.txt
train 740 26844621.txt
train 741 20808492.txt
train 742 28094817.txt
train 743 28866247.txt
train 744 12608733.txt
train 745 22324909.txt
train 746 20661484.txt
train 747 25806950.txt
train 748 24761747.txt
train 749 28393291.txt
train 750 27940149.txt
train 751 16205737.txt
train 752 26151443.txt
train 753 25631989.txt
train 754 17849608.txt
train 755 7002441.txt
train 756 21299356.txt
train 757 26613972.txt
train 758 17542236.txt
train 759 27046572.txt
train 760 30644022.txt
train 761 21423308.txt
train 762 27811193.txt
train 763 29169826.txt
train 764 18726150.txt
train 765 19360647.txt
train 766 19614832.txt
train 767 6853441.txt
train 768 12877405.txt
train 769 6129237.txt
train 770 19633940.txt
train 771 15154076.txt
train 772 2737018.txt
train 773 114508

train 1088 10722961.txt
train 1089 28162768.txt
train 1090 26601658.txt
train 1091 18955747.txt
train 1092 26045508.txt
train 1093 20091940.txt
train 1094 30443699.txt
train 1095 28691518.txt
train 1096 29257937.txt
train 1097 15173555.txt
train 1098 27592327.txt
train 1099 28429618.txt
train 1100 18319196.txt
train 1101 12199134.txt
train 1102 4061763.txt
train 1103 23664828.txt
train 1104 28059555.txt
train 1105 3426129.txt
train 1106 31109252.txt
train 1107 16102953.txt
train 1108 25599888.txt
train 1109 23536211.txt
train 1110 24114230.txt
train 1111 8275903.txt
train 1112 17630501.txt
train 1113 25670416.txt
train 1114 17354572.txt
train 1115 28339164.txt
train 1116 27036065.txt
train 1117 29119522.txt
train 1118 31127088.txt
train 1119 24155303.txt
train 1120 11321200.txt
train 1121 23163236.txt
train 1122 29239739.txt
train 1123 30866976.txt
train 1124 23562581.txt
train 1125 25149176.txt
train 1126 18440037.txt
train 1127 18940023.txt
train 1128 21971779.txt
train 1129 19440131

train 1435 17531206.txt
train 1436 11524898.txt
train 1437 29027066.txt
train 1438 28209607.txt
train 1439 24290390.txt
train 1440 29128349.txt
train 1441 29262471.txt
train 1442 6927697.txt
train 1443 28132125.txt
train 1444 24419683.txt
train 1445 29428574.txt
train 1446 20439799.txt
train 1447 27029898.txt
train 1448 26481385.txt
train 1449 30559470.txt
train 1450 19630898.txt
train 1451 7263933.txt
train 1452 29383400.txt
train 1453 29980900.txt
train 1454 29911305.txt
train 1455 27785461.txt
train 1456 31164130.txt
train 1457 15314136.txt
train 1458 19964971.txt
train 1459 3571738.txt
train 1460 16874561.txt
train 1461 8837904.txt
train 1462 23206665.txt
train 1463 19198918.txt
train 1464 28929285.txt
train 1465 30213685.txt
train 1466 17319242.txt
train 1467 29794887.txt
train 1468 19883285.txt
train 1469 19786040.txt
train 1470 26922967.txt
train 1471 11353400.txt
train 1472 17985266.txt
train 1473 10433407.txt
train 1474 28369738.txt
train 1475 29182941.txt
train 1476 26864158.

...

test 18835 29761854.txt
test 18836 25998072.txt
test 18837 12770674.txt
test 18838 18978390.txt
test 18839 20358271.txt
test 18840 24287372.txt
test 18841 28920569.txt
test 18842 26063446.txt
test 18843 29369670.txt
test 18844 28239772.txt
test 18845 17180714.txt
test 18846 17548752.txt
test 18847 24850670.txt
test 18848 30332294.txt
test 18849 23139830.txt
test 18850 29052463.txt
test 18851 22146274.txt
test 18852 22550944.txt
test 18853 22686868.txt
test 18854 3067904.txt
test 18855 15699309.txt
test 18856 7592145.txt
test 18857 25217958.txt
test 18858 16027737.txt
test 18859 9804340.txt
test 18860 28607477.txt
test 18861 29478156.txt
test 18862 9064496.txt
test 18863 8890510.txt
test 18864 28954536.txt
test 18865 29710337.txt
test 18866 29651681.txt
test 18867 10832774.txt
test 18868 29497988.txt
test 18869 15733271.txt
test 18870 71711.txt
test 18871 19119429.txt
test 18872 25626423.txt
test 18873 28685178.txt
test 18874 27667684.txt
test 18875 28060873.txt
test 18876 8498878.txt
t

test 19182 10832778.txt
test 19183 27233289.txt
test 19184 29360468.txt
test 19185 28493861.txt
test 19186 21182207.txt
test 19187 3308829.txt
test 19188 17018188.txt
test 19189 30128280.txt
test 19190 29216746.txt
test 19191 21213096.txt
test 19192 28030565.txt
test 19193 26152436.txt
test 19194 10868777.txt
test 19195 27322023.txt
test 19196 14999173.txt
test 19197 30250039.txt
test 19198 27895198.txt
test 19199 29774525.txt
test 19200 24136870.txt
test 19201 24187227.txt
test 19202 16553538.txt
test 19203 12587138.txt
test 19204 17610152.txt
test 19205 22977198.txt
test 19206 29069077.txt
test 19207 11840316.txt
test 19208 27671801.txt
test 19209 28256376.txt
test 19210 27343047.txt
test 19211 15662153.txt
test 19212 23821414.txt
test 19213 30392626.txt
test 19214 16239862.txt
test 19215 12919132.txt
test 19216 26471937.txt
test 19217 18369718.txt
test 19218 11353439.txt
test 19219 16119474.txt
test 19220 23880236.txt
test 19221 12959416.txt
test 19222 19886906.txt
test 19223 186565

test 19530 28803940.txt
test 19531 17597540.txt
test 19532 15558079.txt
test 19533 20628440.txt
test 19534 19948568.txt
test 19535 30739049.txt
test 19536 28635242.txt
test 19537 27632384.txt
test 19538 9089122.txt
test 19539 15518914.txt
test 19540 19347776.txt
test 19541 31746178.txt
test 19542 18595288.txt
test 19543 20708861.txt
test 19544 11837403.txt
test 19545 29274915.txt
test 19546 16638734.txt
test 19547 25629966.txt
test 19548 24499184.txt
test 19549 30541423.txt
test 19550 24603775.txt
test 19551 2135638.txt
test 19552 30535103.txt
test 19553 17699123.txt
test 19554 7862687.txt
test 19555 8832772.txt
test 19556 27755291.txt
test 19557 3884641.txt
test 19558 27197696.txt
test 19559 30241574.txt
test 19560 28589496.txt
test 19561 28825309.txt
test 19562 29247487.txt
test 19563 29863400.txt
test 19564 21901837.txt
test 19565 19234777.txt
test 19566 22765335.txt
test 19567 27871670.txt
test 19568 12880797.txt
test 19569 26708327.txt
test 19570 16374669.txt
test 19571 29948531.t

test 19874 27825394.txt
test 19875 21528391.txt
test 19876 28831647.txt
test 19877 29667506.txt
test 19878 29203509.txt
test 19879 22278029.txt
test 19880 30261217.txt
test 19881 19680798.txt
test 19882 18348195.txt
test 19883 22562042.txt
test 19884 9746008.txt
test 19885 30732986.txt
test 19886 30180836.txt
test 19887 10755176.txt
test 19888 8479331.txt
test 19889 20630121.txt
test 19890 18183774.txt
test 19891 28727509.txt
test 19892 26717863.txt
test 19893 15293640.txt
test 19894 28562251.txt
test 19895 23179346.txt
test 19896 30361331.txt
test 19897 21211941.txt
test 19898 11006968.txt
test 19899 28673117.txt
test 19900 19786328.txt
test 19901 16282786.txt
test 19902 28533516.txt
test 19903 29911878.txt
test 19904 9789992.txt
test 19905 14593168.txt
test 19906 15590242.txt
test 19907 30696816.txt
test 19908 29483346.txt
test 19909 27354431.txt
test 19910 27383033.txt
test 19911 15362169.txt
test 19912 28230711.txt
test 19913 25321853.txt
test 19914 8997451.txt
test 19915 26881372.

test 20219 16897400.txt
test 20220 16697302.txt
test 20221 17846198.txt
test 20222 26437875.txt
test 20223 29450820.txt
test 20224 12952554.txt
test 20225 19049604.txt
test 20226 21762713.txt
test 20227 27465318.txt
test 20228 12876158.txt
test 20229 22760338.txt
test 20230 21250997.txt
test 20231 19557195.txt
test 20232 3804959.txt
test 20233 17889735.txt
test 20234 18547459.txt
test 20235 1639438.txt
test 20236 28393290.txt
test 20237 26031922.txt
test 20238 8463201.txt
test 20239 30512467.txt
test 20240 24703983.txt
test 20241 17882540.txt
test 20242 26375827.txt
test 20243 17006862.txt
test 20244 17947290.txt
test 20245 27144642.txt
test 20246 1400698.txt
test 20247 24952201.txt
test 20248 17620483.txt
test 20249 16600060.txt
test 20250 29147839.txt
test 20251 18626761.txt
test 20252 28302197.txt
test 20253 27320472.txt
test 20254 18308764.txt
test 20255 17123170.txt
test 20256 26914612.txt
test 20257 22562209.txt
test 20258 28077731.txt
test 20259 24816870.txt
test 20260 22591180.

# sort the CLAMP output into train and test

In [ ]:
import os
from shutil import copyfile

In [ ]:
ABSTRACT = True

if ABSTRACT:
    directory_train_test = "abstract_train_test"
    INPUT_DIR = "pubmed_abstracts_24860"
else:
    directory_train_test = "full_text_train_test"
    INPUT_DIR = "pubmed_all_fulltext_input"


CORPUS_DIR = os.path.join(directory_train_test, "corpus")

In [ ]:
# create filelist.txt for clamp crf training
with open(os.path.join(directory_train_test, "train_test_split.txt"), "r") as f:
    with open(os.path.join(directory_train_test, "filelist.txt"), "w") as f_out:
        for i, line in enumerate(f, 1):

            filename = line.split("\t")[0].strip()
            xmi_filename = filename.replace(".txt", ".xmi")
            typ = line.split("\t")[1].strip()
            xmi_file_path = os.path.join("corpus", xmi_filename)
        
            if typ == "train" or typ == "devel":
                f_out.write(xmi_file_path + "\t" + "train" + "\n")
            elif typ == "test":
                f_out.write(xmi_file_path + "\t" + "test" + "\n")
            else:
                raise Exception("unknown type", typ)

In [ ]:
# check contents of filelist.txt to make sure correct

train_count = 0
dev_count = 0
test_count = 0

# check
with open(os.path.join(directory_train_test, "filelist.txt"), "r") as f:
    for line in f:
        if "train" in line:
            train_count = train_count + 1
        elif "devel" in line:
            dev_count = dev_count + 1
        elif "test" in line:
            test_count = test_count +1
        else:
            raise Exception("wrong type")
            
print("train_count", train_count)
print("dev_count", dev_count)
print("test_count", test_count)

In [ ]:
# copy train test into different folder
with open(os.path.join(directory_train_test, "train_test_split.txt"), "r") as f:
    
    for i, line in enumerate(f, 1):
        
        filename = line.split("\t")[0].strip()
        typ = line.split("\t")[1].strip()
        
        file_path = os.path.join(INPUT_DIR, filename)

        
        if typ == "train" or typ == "devel":
            new_path = os.path.join(directory_train_test, "train", filename)
        elif typ == "test":
            new_path = os.path.join(directory_train_test, "test", filename)
        else:
            raise Exception("unknown type", typ)
        
        copyfile(file_path, new_path)
        
        #print(file_path + "\t" + new_path)


In [ ]:
# copy LABELLED train test into different folder
with open(os.path.join(directory_train_test, "train_test_split.txt"), "r") as f:
    
    for i, line in enumerate(f, 1):
        
        filename = line.split("\t")[0].strip()
        typ = line.split("\t")[1].strip()
        
        file_path = os.path.join(CORPUS_DIR, filename)

        if typ == "train" or typ == "devel":
            new_path = os.path.join(directory_train_test, "labelled_text", "train", filename)
        elif typ == "test":
            new_path = os.path.join(directory_train_test, "labelled_text", "test", filename)
        else:
            raise Exception("unknown type", typ)
        
        copyfile(file_path, new_path)
        
        #print(file_path + "\t" + new_path)
